# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some test with real teams.

## Imports

In [1]:
from os import makedirs
import json
import random
import pandas as pd
from typing import Callable, Tuple, List
from IPython.display import display, Markdown

from model import FootballSchedulerModel, SymetricScheme
from parse import parse_sol, to_df

In [2]:
countries = ["ARG", "BOL", "BRA", "CHI", "COL", "ECU", "PAR", "PER", "URU", "VEN"]
top_countries = ["ARG", "BRA"]
assert(set(top_countries).issubset(set(countries)))
n = len(countries)

In [3]:
def raffle(countries: list[str]) -> Tuple[Callable[[str], int], Callable[[int], str]]:
	countries_copy = countries.copy()
	random.shuffle(countries_copy)
	return lambda c: countries_copy.index(c), lambda i: countries_copy[i]

def write_raffle(base_path: str, index_of: Callable[[str], int], country_of: Callable[[int], str]):
	with open(f"{base_path}/index_of.json", "w") as f:
		json.dump(dict([(c, index_of(c)) for c in countries]), f)
	with open(f"{base_path}/country_of.json", "w") as f:
		json.dump(dict([(i, country_of(i)) for i in range(n)]), f)

# Represents the random asignment of country <-> id.
index_of, country_of = raffle(countries)

In [4]:
top_teams = [index_of(c) for c in top_countries]

In [5]:
def write_model_and_raffle(
	base_path: str, model: FootballSchedulerModel
):
	makedirs(base_path, exist_ok=True)
	write_raffle(base_path, index_of=index_of, country_of=country_of)
	model.write_problem(f"{base_path}/model.lp")


def assert_optimized(model: FootballSchedulerModel):
	model.optimize()
	assert model.get_obj_value() == 0

def write_solution(base_path: str, model: FootballSchedulerModel):
	sol_path = f"{base_path}/solution.sol"
	model.write_sol(sol_path)

def recover_solution_as_df(base_path:str) -> pd.DataFrame:
	solution_path = f"{base_path}/solution.sol"
	index_of_path = f"{base_path}/index_of.json"
	country_of_path = f"{base_path}/country_of.json"

	sol = parse_sol(solution_path)
	index_of = json.load(open(index_of_path))
	country_of = json.load(open(country_of_path))

	return to_df(
		sol=sol,
		index_of=index_of,
		country_of=country_of,
	)

def write_solution_to_latex(base_path: str, sol_df: pd.DataFrame):
	with open(f"{base_path}/solution_table.tex", "w+") as f:
		f.write(sol_df.to_latex(index=False))

## French Scheme

In [47]:
base_path_french = "output/french"

In [48]:
french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH, top_teams)
write_model_and_raffle(base_path_french, french_model)
assert_optimized(french_model)
print(
    f"Optimization of the french model with top teams took: {french_model.get_solving_time()} seconds"
)
write_solution(base_path_french, french_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french/model.lp
Optimization of the french model with top teams took: 7.86566 seconds


In [54]:
french_df = recover_solution_as_df(base_path_french)
write_solution_to_latex(base_path_french, french_df)
display(Markdown(french_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| ARG    | ECU  | @BRA | @COL | VEN  | @CHI | ARG  | PAR  | @BOL | @URU | BRA  | COL  | @VEN | CHI  | @ARG | @PAR | BOL  | URU  | @ECU |
| BOL    | @VEN | CHI  | @BRA | COL  | @ARG | ECU  | BOL  | @PAR | PER  | @CHI | BRA  | @COL | ARG  | @ECU | @BOL | PAR  | @PER | VEN  |
| BRA    | PAR  | @ARG | @VEN | BRA  | @COL | CHI  | @URU | PER  | @ECU | ARG  | VEN  | @BRA | COL  | @CHI | URU  | @PER | ECU  | @PAR |
| CHI    | @CHI | BOL  | @PAR | ECU  | URU  | @PER | VEN  | @BRA | COL  | @BOL | PAR  | @ECU | @URU | PER  | @VEN | BRA  | @COL | CHI  |
| COL    | ARG  | @URU | @ECU | PAR  | PER  | @BOL | COL  | @VEN | @BRA | URU  | ECU  | @PAR | @PER | BOL  | @COL | VEN  | BRA  | @ARG |
| ECU    | @BOL | COL  | ARG  | @CHI | @ECU | BRA  | @PER | URU  | VEN  | @COL | @ARG | CHI  | ECU  | @BRA | PER  | @URU | @VEN | BOL  |
| PAR    | BRA  | @PAR | PER  | @URU | BOL  | @VEN | @CHI | ECU  | @ARG | PAR  | @PER | URU  | @BOL | VEN  | CHI  | @ECU | ARG  | @BRA |
| PER    | @COL | PER  | URU  | @BOL | VEN  | @PAR | @ECU | ARG  | CHI  | @PER | @URU | BOL  | @VEN | PAR  | ECU  | @ARG | @CHI | COL  |
| URU    | URU  | @ECU | BOL  | @PER | @BRA | COL  | @ARG | CHI  | @PAR | ECU  | @BOL | PER  | BRA  | @COL | ARG  | @CHI | PAR  | @URU |
| VEN    | @PER | VEN  | CHI  | @ARG | PAR  | @URU | BRA  | @COL | BOL  | @VEN | @CHI | ARG  | @PAR | URU  | @BRA | COL  | @BOL | PER  |

## English Scheme

In [50]:
base_path_english = "output/english"

In [51]:
english_model = FootballSchedulerModel(n, SymetricScheme.ENGLISH, top_teams)
write_model_and_raffle(base_path_english, english_model)
assert_optimized(english_model)
print(
    f"Optimization of the english model with top teams took: {english_model.get_solving_time()} seconds"
)
write_solution(base_path_english, english_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/english/model.lp
Optimization of the english model with top teams took: 7.167308 seconds


In [53]:
english_df = recover_solution_as_df(base_path_english)
write_solution_to_latex(base_path_english, english_df)
display(Markdown(english_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| ARG    | CHI  | @URU | VEN  | @ARG | @BRA | ECU  | PAR  | @COL | BOL  | @BOL | @CHI | URU  | @VEN | ARG  | BRA  | @ECU | @PAR | COL  |
| BOL    | @BRA | PER  | ARG  | @ECU | PAR  | @BOL | CHI  | @VEN | @COL | COL  | BRA  | @PER | @ARG | ECU  | @PAR | BOL  | @CHI | VEN  |
| BRA    | @ARG | BRA  | @CHI | PAR  | @VEN | URU  | COL  | @ECU | @PER | PER  | ARG  | @BRA | CHI  | @PAR | VEN  | @URU | @COL | ECU  |
| CHI    | BOL  | @ECU | @URU | PER  | COL  | @CHI | VEN  | @PAR | BRA  | @BRA | @BOL | ECU  | URU  | @PER | @COL | CHI  | @VEN | PAR  |
| COL    | @PER | VEN  | BOL  | @COL | @ECU | ARG  | @URU | BRA  | @PAR | PAR  | PER  | @VEN | @BOL | COL  | ECU  | @ARG | URU  | @BRA |
| ECU    | @VEN | COL  | ECU  | @BOL | @URU | BRA  | @PER | ARG  | CHI  | @CHI | VEN  | @COL | @ECU | BOL  | URU  | @BRA | PER  | @ARG |
| PAR    | ECU  | @PAR | @BRA | CHI  | @ARG | VEN  | @BOL | PER  | URU  | @URU | @ECU | PAR  | BRA  | @CHI | ARG  | @VEN | BOL  | @PER |
| PER    | URU  | @BOL | COL  | @VEN | PER  | @PAR | ECU  | @CHI | @ARG | ARG  | @URU | BOL  | @COL | VEN  | @PER | PAR  | @ECU | CHI  |
| URU    | PAR  | @CHI | @PER | BRA  | BOL  | @COL | @ARG | URU  | @ECU | ECU  | @PAR | CHI  | PER  | @BRA | @BOL | COL  | ARG  | @URU |
| VEN    | @COL | ARG  | @PAR | URU  | CHI  | @PER | @BRA | BOL  | VEN  | @VEN | COL  | @ARG | PAR  | @URU | @CHI | PER  | BRA  | @BOL |

## Inverted scheme

In [6]:
base_path_inverted = "output/inverted"

In [ ]:
inverted_model = FootballSchedulerModel(n, SymetricScheme.INVERTED, top_teams)
write_model_and_raffle(base_path_inverted, inverted_model)
assert_optimized(inverted_model)
print(
    f"Optimization of the inverted model with top teams took: {inverted_model.get_solving_time()} seconds"
)
write_solution(base_path_inverted, inverted_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/inverted/model.lp
Optimization of the inverted model with top teams took: 26.13691 seconds


In [8]:
inverted_df = recover_solution_as_df(base_path_inverted)
write_solution_to_latex(base_path_inverted, inverted_df)
display(Markdown(inverted_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| ARG    | ARG  | @BOL | @PER | VEN  | @URU | CHI  | ECU  | @BRA | PAR  | @PAR | BRA  | @ECU | @CHI | URU  | @VEN | PER  | BOL  | @ARG |
| BOL    | PER  | @VEN | URU  | @PAR | @CHI | ECU  | @BOL | COL  | @ARG | ARG  | @COL | BOL  | @ECU | CHI  | PAR  | @URU | VEN  | @PER |
| BRA    | @PAR | COL  | CHI  | @ARG | @ECU | PER  | BRA  | @URU | VEN  | @VEN | URU  | @BRA | @PER | ECU  | ARG  | @CHI | @COL | PAR  |
| CHI    | @BRA | CHI  | COL  | @URU | PAR  | @BOL | @VEN | ARG  | @ECU | ECU  | @ARG | VEN  | BOL  | @PAR | URU  | @COL | @CHI | BRA  |
| COL    | @CHI | BRA  | PAR  | @COL | @ARG | URU  | PER  | @ECU | @BOL | BOL  | ECU  | @PER | @URU | ARG  | COL  | @PAR | @BRA | CHI  |
| ECU    | @URU | PAR  | ARG  | @CHI | BOL  | @BRA | @COL | VEN  | PER  | @PER | @VEN | COL  | BRA  | @BOL | CHI  | @ARG | @PAR | URU  |
| PAR    | BOL  | @ECU | @VEN | BRA  | @PER | ARG  | URU  | @CHI | @COL | COL  | CHI  | @URU | @ARG | PER  | @BRA | VEN  | ECU  | @BOL |
| PER    | @COL | URU  | @ECU | BOL  | VEN  | @PAR | CHI  | @PER | BRA  | @BRA | PER  | @CHI | PAR  | @VEN | @BOL | ECU  | @URU | COL  |
| URU    | VEN  | @PER | @BOL | ECU  | BRA  | @COL | @ARG | PAR  | @URU | URU  | @PAR | ARG  | COL  | @BRA | @ECU | BOL  | PER  | @VEN |
| VEN    | ECU  | @ARG | @BRA | PER  | COL  | @VEN | @PAR | BOL  | CHI  | @CHI | @BOL | PAR  | VEN  | @COL | @PER | BRA  | ARG  | @ECU |

## Back to Back scheme

In [6]:
base_path_b2b = "output/b2b"

In [7]:
b2b_model = FootballSchedulerModel(n, SymetricScheme.BACK_TO_BACK, top_teams)
write_model_and_raffle(base_path_b2b, b2b_model)
assert_optimized(b2b_model)
print(
    f"Optimization of the b2b model with top teams took: {b2b_model.get_solving_time()} seconds"
)
write_solution(base_path_b2b, b2b_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/b2b/model.lp
Optimization of the b2b model with top teams took: 0.591054 seconds


In [8]:
b2b_df = recover_solution_as_df(base_path_b2b)
write_solution_to_latex(base_path_b2b, b2b_df)
display(Markdown(b2b_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| ARG    | @PAR | PAR  | VEN  | @VEN | @COL | COL  | ECU  | @ECU | @URU | URU  | BRA  | @BRA | BOL  | @BOL | @PER | PER  | CHI  | @CHI |
| BOL    | @ECU | ECU  | @PER | PER  | ARG  | @ARG | @VEN | VEN  | PAR  | @PAR | @CHI | CHI  | URU  | @URU | BOL  | @BOL | BRA  | @BRA |
| BRA    | BOL  | @BOL | URU  | @URU | PER  | @PER | @BRA | BRA  | @VEN | VEN  | COL  | @COL | @PAR | PAR  | @ECU | ECU  | @ARG | ARG  |
| CHI    | @PER | PER  | @ARG | ARG  | ECU  | @ECU | COL  | @COL | CHI  | @CHI | BOL  | @BOL | BRA  | @BRA | @URU | URU  | @PAR | PAR  |
| COL    | @URU | URU  | ECU  | @ECU | @BOL | BOL  | CHI  | @CHI | PER  | @PER | @ARG | ARG  | @VEN | VEN  | PAR  | @PAR | @COL | COL  |
| ECU    | VEN  | @VEN | COL  | @COL | @CHI | CHI  | PAR  | @PAR | @BRA | BRA  | URU  | @URU | @ECU | ECU  | ARG  | @ARG | @BOL | BOL  |
| PAR    | @CHI | CHI  | PAR  | @PAR | BRA  | @BRA | @URU | URU  | ECU  | @ECU | @VEN | VEN  | @ARG | ARG  | @COL | COL  | PER  | @PER |
| PER    | COL  | @COL | @BRA | BRA  | @VEN | VEN  | @ARG | ARG  | @BOL | BOL  | PAR  | @PAR | PER  | @PER | CHI  | @CHI | @URU | URU  |
| URU    | ARG  | @ARG | @BOL | BOL  | URU  | @URU | @PER | PER  | @COL | COL  | @ECU | ECU  | CHI  | @CHI | @BRA | BRA  | VEN  | @VEN |
| VEN    | BRA  | @BRA | @CHI | CHI  | @PAR | PAR  | BOL  | @BOL | ARG  | @ARG | @PER | PER  | @COL | COL  | VEN  | @VEN | ECU  | @ECU |

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] Pyscipopt Docs.